In [1]:
from dotenv import load_dotenv
import os, json
import requests

load_dotenv()
api_key = os.getenv("API_KEY", None)
project_id = os.getenv("PROJECT_ID", None)
ibm_cloud_iam_url = os.getenv("IAM_IBM_CLOUD_URL", None)

creds = {
    "url"    : "https://us-south.ml.cloud.ibm.com",
    "apikey" : api_key
}

params = {
    "decoding_method":"greedy",
    "max_new_tokens":3000,
    "min_new_tokens":1,
    # "temperature":0.1,
    "top_k":50,
    "top_p":1,
     "repetition_penalty": 1.1,
    # "stop_sequences":["```"],
}

In [2]:
# Prepare the payload and headers
payload = {
    "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
    "apikey": api_key
}
headers = {
    'Content-Type': "application/x-www-form-urlencoded"
}

# Make a POST request while ignoring SSL certificate verification
try:
    response = requests.post(f"https://{ibm_cloud_iam_url}/identity/token", data=payload, headers=headers, verify=False)
    
    # Check if the request was successful
    response.raise_for_status()

    # Parse the JSON response
    decoded_json = response.json()
    access_token = decoded_json["access_token"]
    # print(f"Access Token: {access_token}")
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

An error occurred: HTTPSConnectionPool(host='none', port=443): Max retries exceeded with url: /identity/token (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x106761ac0>: Failed to resolve 'none' ([Errno 8] nodename nor servname provided, or not known)"))


In [3]:
import requests

GRANITE_3B_CODE_INSTRUCT = "ibm/granite-3b-code-instruct"
GRANITE_8B_CODE_INSTRUCT = "ibm/granite-8b-code-instruct"
GRANITE_20B_CODE_INSTRUCT = "ibm/granite-20b-code-instruct"
GRANITE_34B_CODE_INSTRUCT = "ibm/granite-34b-code-instruct"

def watsonx_code_gen(prompt,model_id):

    url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation_stream?version=2023-05-29"

    body = {
        "input": prompt,
        "parameters": params,
        "model_id": model_id,
        "project_id": project_id
    }

    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(
        url,
        headers=headers,
        json=body,
        stream=True
    )

    if response.status_code != 200:
        raise Exception("Non-200 response: " + str(response.text))

    answer = ""
    # Stream the response
    for line in response.iter_lines():
        if line:  # Ensure the line is not empty
            decoded_line = line.decode("utf-8").strip()
            
            # Check if the line starts with "data: "
            if decoded_line.startswith("data: "):
                json_data = decoded_line[len("data: "):]  # Remove the "data: " prefix
                
                try:
                    # Attempt to load the JSON data
                    data = json.loads(json_data)
                    generated_text = data.get("results", "")
                    answer += generated_text[0]["generated_text"]
                    # print(generated_text[0]["generated_text"],end="")  # Print or process the generated text as needed
                except json.JSONDecodeError:
                    print("Failed to decode JSON:", json_data)
                except Exception as e:
                    print("An error occurred:", e)

    return answer


In [4]:
sample_code = """import java.util.*;

import com.ibm.j9ddr.vm29.pointer.generated.processor_cpu_load_infoPointer;

public class Main {

    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();
            int[] a = new int[n];
            for(int a_i=0; a_i < n; a_i++){
                a[a_i] = in.nextInt();
            }
        }
    }

    public void calculate(int[] a) {
        int sum = 0;
        for (int i = 0; i < a.length; i++) {
            sum += a[i];
        }
        System.out.println(sum);
    }

    public void print(int[] a) {
        for (int i = 0; i < a.length; i++) {
            System.out.println(a[i]);
        }
    }

    public void sort(int[] a) {
        Arrays.sort(a);
        double median;
        if (a.length % 2 == 0) {
            median = (a[a.length / 2 - 1] + a[a.length / 2]) / 2.0;
        } else {
            median = a[a.length / 2];
        }
    }
}"""

bad_code = """import java.util.*;

import com.ibm.j9ddr.vm29.pointer.generated.processor_cpu_load_infoPointer;

public class Main {

    public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();
    }

    public void calculate(int[] a) {
        int sum = 0;
        for (int i = 0; 
        }
        System.out.println(sum);
    }

    public void print(int[] a) {
        for (int i = 0; System.out.println(a[i]);
        }
    }

    public void sort(int[] a) {
        Arrays.sort(a);
        double me
        if (a.length % 2 == 0) {
           th / 2 - 1] + a[a.length / 2]) / 2.0;
        } else {
            median = a[a.len
        }
    }
}"""

In [ ]:
model_id = GRANITE_8B_CODE_INSTRUCT
language = "java"
documentframework = "javadoc"
testframework = "junit"

generate javadoc

In [ ]:
prompt = f"""Question:
generate {documentframework} document for the {language} provided.
{language}: `{sample_code}`
Answer:
"""

answer = watsonx_code_gen(prompt,model_id)

score, review and improve

In [ ]:
prompt = f"""Question:
you are {language} expert. you need to review the following {language} code. 
-please understand the {language} code provided in detail.
-please review the code in completeness, logic, performance and quality.
-give a score out of 10 for the quality of code.
-please provide constructive suggestion.
<<SYS>>
code: `{sample_code}`
<</SYS>>
Answer:
score and review:"""

suggestion = watsonx_code_gen(prompt,model_id)

prompt = f"""Question:
improve the {language} provided base on the review provided.
-understand the review suggestion in detail.
-adopt the review suggestion if applicable.
{language}: `{sample_code}`
review suggestion: `{suggestion}`
Answer:
improved version:
"""

answer = watsonx_code_gen(prompt,model_id)

fix bad code

In [ ]:
prompt = f"""Question:
fix the {language} provided.
-understand the review suggestion in detail.
-identify if any problem.
-fix the problem if any.
{language}: `{bad_code}`
Answer:
fixed version:
"""

answer = watsonx_code_gen(prompt,model_id)

generate diagram

In [ ]:
prompt = f"""Question:
generate diagram for the {language} code provided.
-understand the review suggestion in detail.
-use simple name for node.
-generate in mermaid.
-dont repeating.
{language}: `{sample_code}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

conversion

In [ ]:
prompt = f"""Question:
rewrite the {language} code provided to python.
-understand the review suggestion in detail.
{language}: `{sample_code}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

generate optimization

In [ ]:
prompt = f"""Question:
generate a improvement suggestion for the {language} code provided.
-understand the {language} code in detail.
-improve object oriented structure.
-improve the naming convension, give more meaningful name.
-improve performance that leverage HPC library if applicable.
-adopt the improvement suggestion.
{language}: `{sample_code}`
Answer:"""

suggestion = watsonx_code_gen(prompt,model_id)

prompt = f"""Question:
generate a improved version base on the {language} code provided.
-understand the {language} code in detail.
-adopt the improvement suggestion.
{language}: `{sample_code}`
suggestion: `{suggestion}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

generate api definition

In [ ]:
prompt = f"""Question:
generate a API definition in swagger for the {language} code provided.
{language}: `{sample_code}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

generate test case

In [ ]:
prompt = f"""Question:
generate Test Cases in {testframework} for the {language} code provided.
-focus on boundary cases.
-focus on exception handling
{language}: `{sample_code}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

generate interface

In [ ]:
prompt = f"""Question:
generate {language} interface version base on the {language} code provided.
{language}: `{sample_code}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

generate build and deployment script

In [ ]:
prompt = f"""Question:
generate build and deployment script for the {language} code provided.
-target environment be liberty on openshift.
{language}: `{sample_code}`
Answer:"""

answer = watsonx_code_gen(prompt,model_id)

HPC optimize

In [ ]:
prompt = f"""Question:
generate a prompt to enhance the {language} code provided.
-the {language} will target for high performance computing.
-consider use CUDA if applicable.
{language}: `{sample_code}`
Answer:"""

suggestion = watsonx_code_gen(prompt,model_id)

prompt = f"""Question:
enhance the {language} code provided base on the suggestion.
-target environment be with Nvidia GPU.
suggestion: `{suggestion}`
{language}: `{sample_code}`
Answer:
enhance version:"""

answer = watsonx_code_gen(prompt,model_id)

code completion

In [ ]:
partial_code = """public static void main(String[] args) {
        try (Scanner in = new Scanner(System.in)) {
            int n = in.nextInt();"""

prompt = f"""Question:
please generate the next few line of {language} code after the code provided provided.
-please understand the name of the function if any.
-please understand the comment if any.
-please generate high quality {language} code.
-end the generation with double newline.
System:
code = `{partial_code}
`
Answer:
next line:```{language}
"""

answer = watsonx_code_gen(prompt,model_id)

generate explanation

In [ ]:
prompt = f"""Question:
Please provide a detailed explanation for the following {language} code:
-ensure have new line after every full stop.
<<SYS>>
code: `{sample_code}`
<</SYS>>
Answer:
explanation:
"""

answer = watsonx_code_gen(prompt,model_id)

style alignment with fewshot learning

In [ ]:
prompt = f"""Question:
generate document for the code. 
code: `void main(){{
    System.out.print("hello");
}}`
Answer:
Here is a documentation generated from your code:
```{language}
/**
 * IBM documentation style of Main, with parameter of args in String[].
 */
public class MainABC {{

    /**
     * The entry point of the program.
     * @param args the command line arguments
     */
    public static void mainDEF(String[] args) {{
        System.out.println("Hello");
    }}
}}
```
Question:
generate document for the code. 
code: `{sample_code}`
Answer:
"""

answer = watsonx_code_gen(prompt,model_id)